In [ ]:
# ==============================================================================
#
# Game Master AI: Learning to Win Tic-Tac-Toe with Q-Learning
#
# This script contains all the code needed to:
# 1. Create a Tic-Tac-Toe game environment.
# 2. Build an AI "Agent" that uses Q-learning to make decisions.
# 3. Train the AI by making it play against itself thousands of times.
# 4. Allow a human player to challenge the fully trained AI.
#
# To run this, save it as a Python file (e.g., `tictactoe_ai.py`) and
# run it from your terminal: `python tictactoe_ai.py`
#
# ==============================================================================

import random
import time
import json # To save and load the AI's "brain"

# ==============================================================================
# Part 1: The Tic-Tac-Toe Game Environment
# ==============================================================================
class TicTacToe:
    """
    This class represents the Tic-Tac-Toe game. It doesn't know about the AI;
    it only knows the rules of the game, how to make moves, and how to check
    for a winner.
    """
    def __init__(self):
        # The board is a list of 9 strings. '_' means the space is empty.
        self.board = ['_'] * 9

    def print_board(self):
        """Prints the current game board to the console in a readable format."""
        print("")
        # We use an f-string to format the board nicely.
        board_str = (
            f" {self.board[0]} | {self.board[1]} | {self.board[2]} \n"
            "-----------\n"
            f" {self.board[3]} | {self.board[4]} | {self.board[5]} \n"
            "-----------\n"
            f" {self.board[6]} | {self.board[7]} | {self.board[8]} \n"
        )
        print(board_str)

    def get_available_moves(self):
        """Returns a list of the indices of all empty spots on the board."""
        return [i for i, spot in enumerate(self.board) if spot == '_']

    def make_move(self, position, player_symbol):
        """Places a player's symbol on the board if the move is valid."""
        if self.board[position] == '_':
            self.board[position] = player_symbol
            return True
        return False

    def check_winner(self, player_symbol):
        """Checks if the specified player has won the game."""
        # All possible winning combinations (rows, columns, diagonals)
        win_conditions = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
            (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
            (0, 4, 8), (2, 4, 6)             # Diagonals
        ]
        for condition in win_conditions:
            # Check if all spots in a winning condition are filled by the player's symbol
            if all(self.board[i] == player_symbol for i in condition):
                return True
        return False

    def is_draw(self):
        """Checks if the game is a draw (i.e., the board is full)."""
        return '_' not in self.board

    def reset(self):
        """Resets the board to an empty state for a new game."""
        self.board = ['_'] * 9

    def get_state(self):
        """
        Returns the current state of the board as a hashable tuple.
        This is crucial because dictionary keys (which we use for the Q-table)
        must be hashable, and lists are not.
        """
        return tuple(self.board)

# ==============================================================================
# Part 2: The AI Agent using Q-Learning
# ==============================================================================
class Agent:
    """
    This class represents the AI player. It uses a Q-learning algorithm
    to learn the best move for any given game state.
    """
    def __init__(self, symbol, learning_rate=0.1, discount_factor=0.9, epsilon=0.9):
        self.symbol = symbol
        self.learning_rate = learning_rate      # alpha: How much we update Q-values based on new info.
        self.discount_factor = discount_factor  # gamma: How much we value future rewards.
        self.epsilon = epsilon                  # The probability of choosing a random move (exploration).
        self.q_table = {}                       # The AI's "brain". Maps (state) -> {action: q_value}.

    def get_q_value(self, state, action):
        """Safely gets the Q-value for a state-action pair, defaulting to 0.0 if not found."""
        return self.q_table.get(state, {}).get(action, 0.0)

    def choose_action(self, state, available_moves):
        """
        Chooses an action using an epsilon-greedy strategy.
        - With probability epsilon, it explores by choosing a random move.
        - Otherwise, it exploits its current knowledge by choosing the best known move.
        """
        if not available_moves:
            return None

        if random.uniform(0, 1) < self.epsilon:
            # Exploration: choose a random available move.
            return random.choice(available_moves)
        else:
            # Exploitation: choose the best move based on Q-values.
            q_values = {move: self.get_q_value(state, move) for move in available_moves}
            max_q = max(q_values.values())
            # If multiple moves have the same max Q-value, choose one randomly.
            best_moves = [move for move, q in q_values.items() if q == max_q]
            return random.choice(best_moves)

    def update_q_table(self, state, action, reward, next_state, next_available_moves):
        """
        Updates the Q-table using the Bellman equation for Q-learning.
        Q(s, a) = Q(s, a) + alpha * [R + gamma * max(Q(s', a')) - Q(s, a)]
        """
        # Get the best Q-value for the next state to calculate the target value.
        max_next_q = 0.0
        if next_available_moves:
            max_next_q = max([self.get_q_value(next_state, move) for move in next_available_moves])

        # Get the current Q-value for the state-action pair.
        old_q_value = self.get_q_value(state, action)

        # Calculate the new Q-value using the formula.
        new_q_value = old_q_value + self.learning_rate * (reward + self.discount_factor * max_next_q - old_q_value)

        # Update the Q-table with the new value.
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_q_value

    def save_q_table(self, file_path):
        """Saves the Q-table to a file."""
        # We need to convert tuple keys to strings to save as JSON
        string_keyed_q_table = {str(k): v for k, v in self.q_table.items()}
        with open(file_path, 'w') as f:
            json.dump(string_keyed_q_table, f)
        print(f"AI brain saved to {file_path}")

    def load_q_table(self, file_path):
        """Loads the Q-table from a file."""
        with open(file_path, 'r') as f:
            string_keyed_q_table = json.load(f)
            # Convert string keys back to tuples
            self.q_table = {eval(k): v for k, v in string_keyed_q_table.items()}
        print(f"AI brain loaded from {file_path}")


# ==============================================================================
# Part 3: The Training Process
# ==============================================================================
def train(agent1, agent2, game, episodes=50000):
    """
    Trains two AI agents by making them play against each other repeatedly.
    """
    print(f"🤖 Starting AI self-play training for {episodes} games...")
    start_time = time.time()

    for episode in range(episodes):
        game.reset()
        current_agent = agent1
        other_agent = agent2

        while True:
            state = game.get_state()
            available_moves = game.get_available_moves()

            if not available_moves: # Game is a draw
                # Last move led to a draw, give a small reward to the agent who made it
                current_agent.update_q_table(prev_state, prev_action, 1, state, [])
                break

            action = current_agent.choose_action(state, available_moves)
            game.make_move(action, current_agent.symbol)
            next_state = game.get_state()
            next_available_moves = game.get_available_moves()

            # Store previous state and action for rewarding later
            prev_state, prev_action = state, action

            if game.check_winner(current_agent.symbol):
                # The current agent won! Give it a high reward.
                current_agent.update_q_table(state, action, 10, next_state, [])
                # The other agent made a move that led to its loss. Give it a penalty.
                other_agent.update_q_table(last_other_agent_state, last_other_agent_action, -10, next_state, [])
                break

            # Store the state and action of the other agent for potential penalty later
            last_other_agent_state = state
            last_other_agent_action = action

            # Switch turns
            current_agent, other_agent = other_agent, current_agent

        # Decay epsilon over time to shift from exploration to exploitation
        if agent1.epsilon > 0.1: agent1.epsilon -= (0.8 / episodes)
        if agent2.epsilon > 0.1: agent2.epsilon -= (0.8 / episodes)

    duration = time.time() - start_time
    print(f"✅ Training complete in {duration:.2f} seconds.")
    # Set epsilon to 0 so the agents only use their learned strategy
    agent1.epsilon = 0
    agent2.epsilon = 0


# ==============================================================================
# Part 4: Playing against the AI
# ==============================================================================
def play_vs_ai(ai_agent, game):
    """
    The main game loop for a human to play against the trained AI.
    """
    print("\n--- Let's Play Tic-Tac-Toe! ---")
    print("You are 'X'. The AI is 'O'.")
    game.reset()

    while True:
        # Human Player's Turn
        game.print_board()
        try:
            move = int(input("Enter your move (a number from 0 to 8): "))
            if move not in game.get_available_moves():
                print("❌ Invalid move. That spot is taken or out of bounds. Try again.")
                continue
        except ValueError:
            print("❌ Invalid input. Please enter a number between 0 and 8.")
            continue

        game.make_move(move, 'X')

        if game.check_winner('X'):
            game.print_board()
            print("🎉 Congratulations! You won!")
            break
        elif game.is_draw():
            game.print_board()
            print("🤝 It's a draw!")
            break

        # AI Agent's Turn
        print("AI is thinking...")
        time.sleep(0.5)
        state = game.get_state()
        available_moves = game.get_available_moves()
        ai_move = ai_agent.choose_action(state, available_moves)
        game.make_move(ai_move, ai_agent.symbol)
        print(f"AI chose to move to spot {ai_move}.")

        if game.check_winner(ai_agent.symbol):
            game.print_board()
            print("😞 The AI wins! It has been trained well.")
            break
        elif game.is_draw():
            game.print_board()
            print("🤝 It's a draw!")
            break

# ==============================================================================
# Main Execution Block
# ==============================================================================
if __name__ == "__main__":

    # Initialize the game environment and AI agent for the opponent ('O')
    tic_tac_toe_game = TicTacToe()
    agent_o = Agent(symbol='O')
    q_table_file = 'tictactoe_q_table.json'

    # Check if a pre-trained AI brain exists
    try:
        agent_o.load_q_table(q_table_file)
        agent_o.epsilon = 0 # Ensure it's in exploitation mode
        print("Loaded pre-trained AI. Ready to play!")
    except FileNotFoundError:
        print("No pre-trained AI found. Starting a new training session.")
        # Create a second agent to train against
        agent_x_trainer = Agent(symbol='X')

        # Train the agents
        train(agent_x_trainer, agent_o, tic_tac_toe_game, episodes=50000)

        # Save the trained brain for future use
        agent_o.save_q_table(q_table_file)

    # Start the game against the trained AI
    play_vs_ai(agent_o, tic_tac_toe_game)

No pre-trained AI found. Starting a new training session.
🤖 Starting AI self-play training for 50000 games...
✅ Training complete in 5.82 seconds.
AI brain saved to tictactoe_q_table.json

--- Let's Play Tic-Tac-Toe! ---
You are 'X'. The AI is 'O'.

 _ | _ | _ 
-----------
 _ | _ | _ 
-----------
 _ | _ | _ 

Enter your move (a number from 0 to 8): 0
AI is thinking...
AI chose to move to spot 7.

 X | _ | _ 
-----------
 _ | _ | _ 
-----------
 _ | O | _ 

Enter your move (a number from 0 to 8): 4
AI is thinking...
AI chose to move to spot 8.

 X | _ | _ 
-----------
 _ | X | _ 
-----------
 _ | O | O 

Enter your move (a number from 0 to 8): 6
AI is thinking...
AI chose to move to spot 1.

 X | O | _ 
-----------
 _ | X | _ 
-----------
 X | O | O 

Enter your move (a number from 0 to 8): 2

 X | O | X 
-----------
 _ | X | _ 
-----------
 X | O | O 

🎉 Congratulations! You won!
